In [54]:
import pandas as pd
import sys
sys.path.insert(0, '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
import pymysql

## Read data from sql server into pandas

**Note**: this method is only for reading dumped .sql file that already live in local server; for usage of that .sql file on another machine, you need to refer this:  
https://blog.csdn.net/spartanfuk/article/details/81368465?depth_1-utm_source=distribute.pc_relevant.none-task

In [55]:
# sql to DF
sql_path = '../Database/dbs/'
sql_file = 'yelp_test.sql'
sql = open(sql_path + sql_file, 'r', encoding='utf8')
sqltxt = sql.readlines()
sql.close()
sql = "".join(sqltxt)  # list to str

In [56]:
# build DB connections
sql_conn = pymysql.connect(host="localhost",
                      user="root",
                      password="skrskr220",
                      db="yelp_info_db",
                      charset='utf8')
yelp_df = pd.read_sql("SELECT * FROM YELP_SF;", sql_conn)

In [57]:
print(yelp_df["Amenities"][0])

'[('Yes', 'Stroller at Table'), ('Yes', 'Changing Tables'), ('Yes', 'High Chairs'), ('Yes', 'Happy Hour Specials'), ('Yes', 'Vegetarian Options'), ('No', 'Takes Reservations'), ('No', 'Delivery'), ('No', 'Take-out'), ('Yes', 'Accepts Credit Cards'), ('No', 'Accepts Google Pay'), ('No', 'Accepts Cryptocurrency'), ('Street, Validated', 'Parking'), ('No', 'Bike Parking'), ('Yes', 'Wheelchair Accessible'), ('Beer &amp; Wine Only', 'Alcohol'), ('Yes', 'Good For Happy Hour'), ('No', 'Outdoor Seating'), ('No', 'Wi-Fi'), ('No', 'Has TV'), ('No', 'Dogs Allowed'), ('Yes', 'Waiter Service'), ('No', 'Caters'), ('No', 'By Appointment Only'), ('Yes', 'Gender Neutral Restrooms'), ('Yes', 'Open to All')]'


In [58]:
yelp_df

,Address,Amenities,Category,Highlights,Name,Open_hours,Phone,Price,Rating,Reviews
0,"'1735 Polk St,San Francisco,CA,94109'","'[('Yes', 'Stroller at Table'), ('Yes', 'Chang...",'','','Dokkaebier',"'Mon,Closed,Tue,Closed,Wed,4:00 pm - 12:00 am,...",'(415) 625-3203','','5.0','12'
1,"'2125 Fillmore St,San Francisco,CA,94115'","'[('Yes', 'High Chairs'), ('Yes', 'Vegan Optio...","'Mediterranean,Turkish'",'','Troya',"'Mon,11:00 am - 9:30 pm,Tue,11:00 am - 9:30 pm...",'(415) 563-1000','$11-30','4.0','581'
2,"'974 Valencia St,San Francisco,CA,94110'","'[('Yes', 'Vegan Options'), ('Yes', 'Takes Res...","'Mexican,Tapas/Small Plates'",'','Loló',"'Mon,5:30 pm - 11:00 pm,Tue,5:30 pm - 11:00 pm...",'(415) 643-5656','$11-30','4.0','2192'
3,"'11 Division St,San Francisco,CA,94103'","'[('Yes', 'Vegetarian Options'), ('No', 'Takes...","'Dim Sum,Asian Fusion'",'','Dumpling Time',"'Mon,11:00 am - 2:30 pm,5:00 pm - 9:30 pm,Tue,...",'(415) 525-4797','$11-30','4.0','1833'
4,"'4101 Judah St,San Francisco,CA,94122'","'[('Yes', 'High Chairs'), ('Yes', 'Vegan Optio...","'Seafood,Vietnamese'",'','Thanh Long',"'Mon,Closed,Tue,5:00 pm - 9:00 pm,Wed,5:00 pm ...",'(415) 665-1146','$31-60','4.0','2701'
5,"'1120-30 4th St,San Francisco,CA,94158'","'[('Yes', 'Vegan Options'), ('Yes', 'Vegetaria...",'Ethiopian','','Tadu Ethiopian Kitchen',"'Mon,11:30 am - 9:30 pm,Tue,11:30 am - 9:30 pm...",'(415) 655-9344','$11-30','4.5','249'
6,"'Pier 39,Ste A-202,San Francisco,CA,94133'","'[('Yes', 'High Chairs'), ('Yes', 'Kids Activi...","'Seafood,Bars'","'Happy hour specials,Locally sourced ingredien...",'Fog Harbor Fish House',"'Mon,11:00 am - 9:00 pm,Tue,11:00 am - 9:00 pm...",'(415) 421-2442','$11-30','4.0','6041'
7,"'579 Castro St,San Francisco,CA,94114'","'[('Yes', 'High Chairs'), ('No', 'Takes Reserv...",'Seafood','','Anchor Oyster Bar',"'Mon,11:30 am - 10:00 pm,Tue,11:30 am - 10:00 ...",'(415) 431-3990','$11-30','4.5','1514'
8,"'3251 20th Ave,Space 184,San Francisco,CA,94132'","'[('Yes', 'Stroller at Table'), ('Yes', 'High ...","'Japanese,Noodles'",'','Marugame Udon',"'Mon,11:00 am - 10:00 pm,Tue,11:00 am - 10:00 ...",'(415) 680-1280','$11-30','4.0','1684'
9,"'1965 Al Scoma Way,San Francisco,CA,94133'","'[('Yes', 'Kids Activities Nearby'), ('Yes', '...",'Seafood','','Scoma's Restaurant',"'Mon,12:00 pm - 9:00 pm,Tue,12:00 pm - 9:00 pm...",'(415) 771-4383','$31-60','4.0','3759'


## Playground for general data analysis